In [129]:
import numpy as np

from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline, PipelineModel

from pyspark.sql.functions import *
from pyspark.sql.types import *


sc = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName('FinalProject').getOrCreate()

# Get sparkcontext from 
sqlContext = SQLContext(sc)


In [9]:
# ERASING ALL JSON FROM LOCAL

df_business = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_business.json')
df_reviews  = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_review.json')
df_users  = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_user.json')


# FILTERING DATASET TO VANCOUVER

df_business_VA = df_business.select('business_id', 'name',  'address', 'city', \
                                      'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories') \
                                .filter( (df_business.city == 'Vancouver') & (df_business.categories.contains('Restaurants') ) )

business_count = df_business_VA.count()
print(f'MA Business count: {business_count}')

# Sampling data to run on local
sampled_business_MA = df_business_VA.sample(0.01)


# FETCHING USER REVIEWS FOR FILTERED BUSINESES

df_reviews_MA = df_reviews.join(sampled_business_MA, on = 'business_id', how = 'inner') \
                          .select(df_reviews.business_id, df_reviews.user_id, df_reviews.review_id, df_reviews.stars)
df_reviews_count = df_reviews_MA.count()
print(f'Business Reviews count: {df_reviews_count}')



# FETCHING USERS
df_users_MA = df_users.join(df_reviews_MA, on = 'user_id', how = 'inner') \
                          .select(df_users.user_id, df_users.name, df_users.review_count, df_users.yelping_since, \
                                  df_users.useful, df_users.funny , df_users.cool , df_users.fans , df_users.average_stars)
df_users_MA.show(2)

df_users_MA_count = df_users_MA.count()
print(f'Users count: {df_users_MA_count}')




# WRITRING THE DFs into paraquet

sampled_business_MA.coalesce(1).write.parquet('New_Small_Datasets/small_business_datset')
df_reviews_MA.coalesce(1).write.parquet('New_Small_Datasets/small_reviews_dataset')
df_users_MA.coalesce(1).write.parquet('New_Small_Datasets/small_users_dataset')




MA Business count: 5026


Business Reviews count: 16472


+--------------------+----+------------+-------------------+------+-----+----+----+-------------+
|             user_id|name|review_count|      yelping_since|useful|funny|cool|fans|average_stars|
+--------------------+----+------------+-------------------+------+-----+----+----+-------------+
|hCAuMs7R7FFh4gbkM...|Chon|         370|2009-12-25 23:15:20|   319|  127| 154|  14|         3.77|
|OyLO6fl4st6r_YX-L...|Mish|          59|2009-06-23 22:52:52|    47|   18|  35|   2|          3.8|
+--------------------+----+------------+-------------------+------+-----+----+----+-------------+
only showing top 2 rows



Users count: 16472


In [20]:
# Read data generated from above process
business_df = spark.read.parquet('New_Small_Datasets/small_business_datset')
business_count = business_df.count()

reviews_df = spark.read.parquet('New_Small_Datasets/small_reviews_dataset')
reviews_count = reviews_df.count()

users_df = spark.read.parquet('New_Small_Datasets/small_users_dataset')
users_count = users_df.count()

print(f'Business COunt: {business_count}')
print(f'Reviews COunt: {reviews_count}')
print(f'Users COunt: {users_count}')

Business COunt: 256
Reviews COunt: 16472
Users COunt: 16472


### Creating ALS Model

In [89]:
busineses_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "restaurants").option("user", "root")\
    .option("password", "Nanu1996").load()

users_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "users").option("user", "root")\
    .option("password", "Nanu1996").load()

reviews_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "reviews").option("user", "root")\
    .option("password", "Nanu1996").load()

In [91]:
reviews_sql_df.show(5)

+-----+-------------+-------+-----+--------------------+--------------------+
|   id|restaurant_id|user_id|stars|          created_at|          updated_at|
+-----+-------------+-------+-----+--------------------+--------------------+
|16473|         2731|  11079|  1.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16474|         2747|   8720|  1.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16475|         2753|  11903|  5.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16476|         2740|   4151|  1.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16477|         2732|   3695|  3.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
+-----+-------------+-------+-----+--------------------+--------------------+
only showing top 5 rows



In [125]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import to_json,col


reviews_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "reviews").option("user", "root")\
    .option("password", "Nanu1996").load()


(train, test) = reviews_sql_df.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(userCol="user_id", itemCol="restaurant_id", ratingCol="stars", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)



# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))



cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)




#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel




# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())



Num models to be tested:  16
CrossValidator_061c47aea2bb


<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.01


In [126]:


# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)


nrecommendations = best_model.recommendForAllUsers(20)
nrecommendations_json = nrecommendations.select('user_id', to_json(col('recommendations'))).withColumnRenamed("to_json(recommendations)","recommendations")

nrecommendations_json.write.option("truncate", "true").format('jdbc').options(
      url='jdbc:mysql://localhost/food_hunter_development',
      driver='com.mysql.jdbc.Driver',
      dbtable='user_recommendations',
      user='root',
      password='Nanu1996').mode("overwrite").save()



1.4273658210501932


In [132]:
test_predictions.show()

+-----+-------------+-------+-----+--------------------+--------------------+----------+
|   id|restaurant_id|user_id|stars|          created_at|          updated_at|prediction|
+-----+-------------+-------+-----+--------------------+--------------------+----------+
|16530|         2731|   9376|  3.0|2022-04-03 07:37:...|2022-04-03 07:37:...| 3.3832917|
|16884|         2760|   5803|  5.0|2022-04-03 07:37:...|2022-04-03 07:37:...| 3.1077015|
|17731|         2743|  11458|  5.0|2022-04-03 07:37:...|2022-04-03 07:37:...|   3.78815|
|19991|         2795|  13289|  4.0|2022-04-03 07:37:...|2022-04-03 07:37:...| 3.3819737|
|25956|         2903|  11858|  5.0|2022-04-03 07:37:...|2022-04-03 07:37:...|  3.355189|
|26237|         2893|   9376|  4.0|2022-04-03 07:37:...|2022-04-03 07:37:...|  3.186566|
|27619|         2903|   6654|  2.0|2022-04-03 07:37:...|2022-04-03 07:37:...| 2.7607985|
|28586|         2914|   5803|  4.0|2022-04-03 07:37:...|2022-04-03 07:37:...| 3.9960735|
|18064|         2783|

In [145]:
nrecommendations_json.show(20, 60)

+-------+------------------------------------------------------------+
|user_id|                                             recommendations|
+-------+------------------------------------------------------------+
|   3566|[{"restaurant_id":2957,"rating":4.999135},{"restaurant_id...|
|   3582|[{"restaurant_id":2861,"rating":4.9140463},{"restaurant_i...|
|   3587|[{"restaurant_id":2791,"rating":4.999191},{"restaurant_id...|
|   3691|[{"restaurant_id":2855,"rating":1.3261416},{"restaurant_i...|
|   3706|[{"restaurant_id":2945,"rating":1.0624769},{"restaurant_i...|
|   3725|[{"restaurant_id":2962,"rating":2.1796079},{"restaurant_i...|
|   3761|[{"restaurant_id":2815,"rating":2.2087924},{"restaurant_i...|
|   3790|[{"restaurant_id":2945,"rating":2.8243268},{"restaurant_i...|
|   3794|[{"restaurant_id":2909,"rating":3.9905057},{"restaurant_i...|
|   3834|[{"restaurant_id":2791,"rating":5.035022},{"restaurant_id...|
|   3997|[{"restaurant_id":2827,"rating":3.9992065},{"restaurant_i...|
|   40